In [82]:
import pandas as pd
import numpy as np
from datetime import datetime as DateTime, timedelta as TimeDelta

In [83]:
docuConfirma = pd.read_excel("Covid Chile V2 PARA PRUEBAS.xlsx",  sheet_name="COVID_CL_CONFIRMA")
docuMuertes = pd.read_excel("Covid Chile V2 PARA PRUEBAS.xlsx",  sheet_name="COVID_CL_MUERTE")
docuFinal = pd.read_excel("Covid Chile V2 PARA PRUEBAS.xlsx",  sheet_name="FINAL")

In [84]:
docuConfirma.dropna(axis = 0, how = 'all', inplace = True)
if(docuConfirma.columns[0] == 'Unnamed: 0'):
    docuConfirma.columns = docuConfirma.iloc[0]
    docuConfirma = docuConfirma.iloc[1:,].reindex()
    
docuMuertes.dropna(axis = 0, how = 'all', inplace = True)
if(docuMuertes.columns[0] == 'Unnamed: 0'):
    docuMuertes.columns = docuMuertes.iloc[0]
    docuMuertes = docuMuertes.iloc[1:,].reindex()

In [119]:
#dfFecha = pd.DataFrame({"Fecha" : docuConfirma["Fecha"]})
#res = dfFecha.groupby(['Fecha']).sum().reset_index()
fechaMinima = docuConfirma.Fecha.min()
fechaMaxima = docuConfirma.Fecha.max() + TimeDelta(days=5)

listaFechas = []
listaFechas = pd.date_range(start=fechaMinima,end=fechaMaxima)

df3 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"], "Codigo comuna" : docuConfirma["ID_Comuna"]})
res2 = df3.groupby(["Region","Comuna", "Codigo comuna"]).sum().reset_index()

In [120]:
lista = []
for Comuna in res2["Comuna"]:
    for Fecha in listaFechas:
        lista.append(Fecha)
        
res3 = pd.concat([res2]*len(listaFechas))
res3['Fecha'] = lista
res3['Casos Diarios'] = 0
res3['Fallecidos Diarios'] = 0

In [121]:
df4 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"],"Codigo comuna" : docuConfirma["ID_Comuna"] ,"Fecha" : docuConfirma["Fecha"], "Casos Diarios" : 0})

resConfirma4 = df4.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resConfirma4.insert(4, "Fallecidos Diarios", 0, True)
resConfirma5 = pd.concat([ res3 ,  resConfirma4 ])
resConfirma6 = resConfirma5.groupby(['Region','Comuna','Codigo comuna','Fecha']).sum().reset_index()

In [122]:
df5 = pd.DataFrame({"Region" : docuMuertes["Region"], "Comuna" : docuMuertes["Comuna"], "Codigo comuna" : docuConfirma["ID_Comuna"] ,"Fecha" : docuMuertes["Fecha"], "Fallecidos Diarios" : 0})
resMuerte4 = df5.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resMuerte4.insert(3, "Casos Diarios", 0, True) 
resTotal = pd.concat([ resConfirma6 ,  resMuerte4])
resTotal = resTotal.groupby(['Region','Comuna','Codigo comuna','Fecha']).sum().reset_index()

In [123]:
resTotal["Muertes"] = resTotal['Fallecidos Diarios']
for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Muertes'] = resTotal.loc[i, 'Muertes'] + resTotal.loc[i-1, 'Muertes']

In [124]:
resTotal["Casos Acumulados"] = resTotal['Casos Diarios']
for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Casos Acumulados'] = resTotal.loc[i, 'Casos Acumulados'] + resTotal.loc[i-1, 'Casos Acumulados']

In [125]:
resTotal

,Region,Comuna,Codigo comuna,Fecha,Casos Diarios,Fallecidos Diarios,Muertes,Casos Acumulados
0,Antofagasta,Antofagasta,2101,2020-03-03,0,0,0,0
1,Antofagasta,Antofagasta,2101,2020-03-04,0,0,0,0
2,Antofagasta,Antofagasta,2101,2020-03-05,0,0,0,0
3,Antofagasta,Antofagasta,2101,2020-03-06,0,0,0,0
4,Antofagasta,Antofagasta,2101,2020-03-07,0,0,0,0
...,...,...,...,...,...,...,...,...
21007,Ñuble,Ñiquén,16303,2020-05-08,0,0,1,3
21008,Ñuble,Ñiquén,16303,2020-05-09,0,0,1,3
21009,Ñuble,Ñiquén,16303,2020-05-10,0,0,1,3
21010,Ñuble,Ñiquén,16303,2020-05-11,0,0,1,3


In [126]:
resTotal.to_csv("FINAL.csv", index=False)